In [113]:
import feedparser
import pandas as pd
from datetime import datetime, timedelta
import ssl
from bs4 import BeautifulSoup
import warnings

warnings.filterwarnings("ignore")

# SSL sertifika doğrulama sorununu çözmek için
if hasattr(ssl, '_create_unverified_context'):
    ssl._create_default_https_context = ssl._create_unverified_context

In [114]:
links = ["https://bair.berkeley.edu/blog/feed.xml",
        "https://feeds.feedburner.com/nvidiablog",
        "https://www.oreilly.com/radar/feed/",
        "https://www.microsoft.com/en-us/research/feed/",
        "https://www.sciencedaily.com/rss/computers_math/artificial_intelligence.xml",
        "https://research.facebook.com/feed/",
        "https://openai.com/news/rss.xml",
        "https://deepmind.google/blog/feed/basic/",
        ]

In [115]:
def fetch_feed(links):
    entries = {"Title": [], "Link": [], "Published": [], "Description": []}
    
    try:
        # Linklerin üzerinden geçilir
        for link in links:
            feed = feedparser.parse(link)
            
            # Feeddeki her bir girişi işler
            for entry in feed.entries:
                entries["Title"].append(entry.get("title", "No Title"))
                entries["Link"].append(entry.get("link", "No Link"))
                entries["Published"].append(entry.get("published", "No Date"))
                entries["Description"].append(entry.get("description", "No Description"))
                
    except Exception as e:
        # Hata durumunda hata mesajını yazdır
        print(f"An error occurred: {e}")

    # DataFrame oluşturuluyor
    df = pd.DataFrame(entries)
    return df

In [116]:
df = fetch_feed(links)

In [117]:
def extract_and_clean_data(df):
    try:
        # Regex pattern for date extraction
        pattern = r'(\d{2} \w{3} \d{4})'

        # Apply regex to the 'Published' column and extract the date
        df['date'] = df['Published'].str.extract(pattern)

        # Convert the extracted date to datetime
        df['date'] = pd.to_datetime(df['date'], format='%d %b %Y')

        # Drop the original 'Published' column
        df.drop(columns=['Published'], inplace=True)

        # Get today's date and calculate the date 7 days ago
        today = datetime.now()
        seven_days_ago = today - timedelta(days=7)

        # Filter the rows within the last 7 days
        df_last_seven_days = df[(df['date'] >= seven_days_ago) & (df['date'] <= today)]

        # Sort by date in descending order
        df_last_seven_days.sort_values(by='date', ascending=False, inplace=True)

        # Function to clean HTML tags
        def clean_html(text):
            try:
                soup = BeautifulSoup(text, "html.parser")
                return soup.get_text()
            except Exception as e:
                print(f"Error cleaning HTML: {e}")
                return text

        # Apply the HTML cleaning function and shorten descriptions to 500 characters
        df_last_seven_days['Description'] = df_last_seven_days['Description'].apply(lambda x: clean_html(x)[:500])

        # Remove newline characters from the 'Description' column
        df_last_seven_days["Description"] = df_last_seven_days["Description"].str.replace("\n", "")

        return df_last_seven_days

    except Exception as e:
        print(f"An error occurred while processing the data: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of error


In [118]:
df_last_seven_days = extract_and_clean_data(df)

In [119]:
df_last_seven_days


,Title,Link,Description,date
0,Defending against Prompt Injection with Struct...,http://bair.berkeley.edu/blog/2025/04/11/promp...,Recent advances in Large Language Models (LLMs...,2025-04-11
10,National Robotics Week — Latest Physical AI Re...,https://blogs.nvidia.com/blog/national-robotic...,Check back here throughout the week to learn t...,2025-04-11
11,Beyond CAD: How nTop Uses AI and Accelerated C...,https://blogs.nvidia.com/blog/ntop-computer-ai...,"As a teenager, Bradley Rothenberg was obsessed...",2025-04-11
12,Myth and Mystery Await: GeForce NOW Brings ‘So...,https://blogs.nvidia.com/blog/geforce-now-thur...,Get ready to explore the Deep South. South of ...,2025-04-10
13,(AI)ways a Cut Above: GeForce RTX 50 Series Ac...,https://blogs.nvidia.com/blog/studio-rtx-ai-ga...,"As AI-powered tools continue to evolve, NVIDIA...",2025-04-10
14,NVIDIA Celebrates Partners of the Year Advanci...,https://blogs.nvidia.com/blog/nvidia-partner-n...,NVIDIA this week recognized the contributions ...,2025-04-10
109,BrowseComp: a benchmark for browsing agents,https://openai.com/index/browsecomp,BrowseComp: a benchmark for browsing agents.,2025-04-10
28,Debug-gym: an environment for AI coding tools ...,https://www.microsoft.com/en-us/research/blog/...,Developers spend a lot of time debugging code....,2025-04-10
39,Hopping gives this tiny robot a leg up,https://www.sciencedaily.com/releases/2025/04/...,"A hopping, insect-sized robot can jump over ga...",2025-04-09
110,OpenAI Pioneers Program,https://openai.com/index/openai-pioneers-program,Advancing model performance and real world eva...,2025-04-09


In [112]:
df_last_seven_days.to_excel('news.xlsx', index=False)

In [121]:
links = {"https://bair.berkeley.edu/blog/feed.xml": "The Berkeley Artificial Intelligence Research Blog",
        "https://feeds.feedburner.com/nvidiablog": "NVDIA Blog",
        "https://www.microsoft.com/en-us/research/feed/": "Microsoft Research",
        "https://www.sciencedaily.com/rss/computers_math/artificial_intelligence.xml": "Science Daily",
        "https://research.facebook.com/feed/" : "META Research",
        "https://openai.com/news/rss.xml": "OpenAI News",
        "https://deepmind.google/blog/feed/basic/" : "Google DeepMind Blog<",
    }

In [129]:
for i,j in links.items():
    print(i,j)

https://bair.berkeley.edu/blog/feed.xml The Berkeley Artificial Intelligence Research Blog
https://feeds.feedburner.com/nvidiablog NVDIA Blog
https://www.microsoft.com/en-us/research/feed/ Microsoft Research
https://www.sciencedaily.com/rss/computers_math/artificial_intelligence.xml Science Daily
https://research.facebook.com/feed/ META Research
https://openai.com/news/rss.xml OpenAI News
https://deepmind.google/blog/feed/basic/ Google DeepMind Blog<


In [127]:
list(links.keys())

['https://bair.berkeley.edu/blog/feed.xml',
 'https://feeds.feedburner.com/nvidiablog',
 'https://www.microsoft.com/en-us/research/feed/',
 'https://www.sciencedaily.com/rss/computers_math/artificial_intelligence.xml',
 'https://research.facebook.com/feed/',
 'https://openai.com/news/rss.xml',
 'https://deepmind.google/blog/feed/basic/']